# Data loading & Pre-Processing

In [ ]:
import pandas as pd
import dask.dataframe as dd
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
#Setting current directory

os.chdir("..")

In [ ]:
#Setting directory for input & output files

path=os.getcwd()
input_files = path+'/Input_files/'
output_files = path+'/Output/'

Column legends

D_* = Delinquency variables
S_* = Spend variables
P_* = Payment variables
B_* = Balance variables
R_* = Risk variables

In [ ]:
amex_tx_data = dd.read_csv(input_files+'train_data.csv') 

In [ ]:
%%time

amex_tx_data = amex_tx_data.compute()

In [ ]:
op_labels = dd.read_csv(input_files+'train_labels.csv',assume_missing=True)

In [ ]:
%%time
op_labels = op_labels.compute()

In [ ]:
print(amex_tx_data.shape)
amex_tx_data = amex_tx_data.merge(op_labels, how='left', on='customer_ID')
print(amex_tx_data.shape)

In [ ]:
data_amx = amex_tx_data.copy()

In [ ]:
%%time
data_amx['target'].value_counts() #checking if dataset is balanced or not

In [ ]:
data_amx1 = data_amx.groupby('customer_ID')#grouped on 'customer id'

In [ ]:
%%time

data_amx2 = data_amx1.tail(1) #selected latest transaction records

In [ ]:
data_amx2.drop(columns=['S_2'],inplace=True)
data_amx2

In [ ]:
data_amx2['target'].value_counts()

In [ ]:
data_amx2.describe()

# Encoding features

There is ordered categorical data in columns 'D_63' & 'D_64', so we will encode those columns using ordinal encoding

Ordered categorical means it will indicate range or intensity.

for ex: low, medium, high

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
oe = OrdinalEncoder()

In [ ]:
data_amx2['D_63'] = oe.fit_transform(data_amx2[['D_63']])
data_amx2['D_64'] = oe.fit_transform(data_amx2[['D_64']])

In [ ]:
data_amx2.fillna(0,inplace=True)

In [ ]:
x = data_amx2.iloc[:,1:-1]
y = data_amx2.iloc[:,-1]

In [ ]:
from imblearn.over_sampling import SMOTE
# Resampling the minority class. The strategy can be changed as required.
sm = SMOTE(sampling_strategy='minority', random_state=42)

In [ ]:
x_s, y_s = sm.fit_resample(x,y)

# Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
data_amx2.corr()

In [ ]:
test_df=pd.DataFrame(columns=['Col_Name','f_Value'])
test_df['Col_Name'] = x_s.columns
test_df['f_Value']= f_classif(x_s,y_s)[0]
test_df

In [ ]:
feature_rank = SelectKBest(score_func=f_classif, k=5)
ordered_features = feature_rank.fit(x_s,y_s)

In [ ]:
ordered_features.get_feature_names_out()

In [ ]:
x_s = x_s[ordered_features.get_feature_names_out()]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_s,y_s,test_size=0.3,random_state=0)

In [ ]:
#Function to predict result as per called/imported model

def predictor_f(model):
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print('Accuracy Score: ',accuracy_score(y_test,y_pred))
    print('F1_Score: ',f1_score(y_test,y_pred))
    print('Precision Score: ',precision_score(y_test,y_pred))
    print('Recall Score: ',recall_score(y_test,y_pred))

# Linear models


### Logistic Regression

In [ ]:
from sklearn.linear_model import  LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
#Result with 8 best columns

predictor_f(lr)

### Passive Agressive Classifier

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [ ]:
pa = PassiveAggressiveClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(pa)

### Ridge Classifier

In [ ]:
from sklearn.linear_model import RidgeClassifier

In [ ]:
rg = RidgeClassifier()

In [ ]:
#Result with 8 best columns
predictor_f(rg)

### Stochastic Gradient Descent Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(sgd)

# Tree Classifiers

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(dtc)

### Extra Tree Classifier

In [ ]:
from sklearn.tree import ExtraTreeClassifier

In [ ]:
et = ExtraTreeClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(et)

# Ensemble Classifiers

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(rfc)

### Gradient boosting 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(gbc)

### Adaboost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ad = AdaBoostClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(ad)

### Bagging Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
#Result with 8 best columns

bg = BaggingClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(bg)

### Histogram based Gradient boosting

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
hgb = HistGradientBoostingClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(hgb)

### Extra Trees classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
ets = ExtraTreesClassifier()

In [ ]:
#Result with 8 best columns

predictor_f(ets)

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB()

In [ ]:
#Result with 8 best columns

predictor_f(nb)